# Camada Gold - Delivery Delay & Customer Satisfaction

> **POC:** E-commerce Delivery Analytics  
> **Objetivo:** Diagnóstico de atraso logístico e impacto na satisfação do cliente  

Este notebook documenta a construção da **camada Gold** utilizada na POC de **Analytics Diagnóstico**, a partir da base de dados previamente curada.

O foco desta etapa é consolidar, em nível de pedido, informações operacionais, financeiras e de experiência do cliente em um dataset único, estruturado para suportar análises causais e diagnósticas sobre o impacto do atraso logístico na satisfação do consumidor.

---

## Contexto

Os dados utilizados neste notebook são derivados diretamente da camada de **curadoria (Silver)**, construída no notebook [`01_curadoria_sql.ipynb`](notebooks/01_curadoria_sql.ipynb).

Toda a lógica de ingestão, padronização, validação e agregações intermediárias já foi realizada anteriormente.  
Neste notebook, **não há tratamento de dados brutos** — apenas consolidação analítica orientada à geração de insights.

O dataset original tem como base o **Olist E-commerce Dataset**, amplamente utilizado em estudos de analytics e ciência de dados, previamente organizado em camadas Bronze e Silver no DuckDB.

---

## Objetivo da Camada Gold

A camada Gold desta POC foi projetada para:

- Integrar dados logísticos, financeiros e de satisfação do cliente
- Criar métricas derivadas e *flags analíticas* prontas para exploração
- Permitir análises causais entre atraso na entrega e avaliação do cliente
- Servir como base única para a etapa de **Análise Exploratória Guiada (EDA)** e relatório executivo

O resultado final desta etapa é a view **`gold_delay_satisfaction`**, com granularidade de **1 linha por pedido**, adequada para análises analíticas e diagnósticas.

## Conexão e Verificação Inicial

Nesta seção são realizadas a importação das bibliotecas necessárias, a conexão com o banco de dados DuckDB e uma verificação inicial das tabelas e views disponíveis no ambiente analítico.

In [1]:
import sys; sys.path.insert(0, "..")
from src.paths import PROCESSED_DATA
import duckdb

In [2]:
con = duckdb.connect(database=str(PROCESSED_DATA), read_only=False)
con.execute("SELECT 1").fetchone()

(1,)

In [3]:
# listar tabelas e views
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,category_translation,BASE TABLE
1,main,customers,BASE TABLE
2,main,geolocation,BASE TABLE
3,main,order_items,BASE TABLE
4,main,order_payments,BASE TABLE
5,main,order_reviews,BASE TABLE
6,main,orders,BASE TABLE
7,main,products,BASE TABLE
8,main,sellers,BASE TABLE
9,main,curated_category_translation,VIEW


In [4]:
con.execute("""
SELECT
  (SELECT COUNT(*) FROM curated_orders) AS total_orders,
  (SELECT COUNT(*) FROM curated_reviews_by_order) AS orders_with_reviews
""").df()

,total_orders,orders_with_reviews
0,99441,98673


In [5]:
con.execute("DESCRIBE curated_reviews_by_order;").df()

,column_name,column_type,null,key,default,extra
0,order_id,VARCHAR,YES,None,None,None
1,review_id,VARCHAR,YES,None,None,None
2,review_score,BIGINT,YES,None,None,None
3,has_comment,INTEGER,YES,None,None,None
4,is_detractor,INTEGER,YES,None,None,None
5,is_promoter,INTEGER,YES,None,None,None
6,review_creation_ts,TIMESTAMP,YES,None,None,None
7,review_answer_ts,TIMESTAMP,YES,None,None,None
8,review_date,DATE,YES,None,None,None
9,answer_date,DATE,YES,None,None,None


## Construção da Camada Gold — `gold_delay_satisfaction`

A view `gold_delay_satisfaction` consolida múltiplas dimensões do negócio em nível de pedido, combinando:

- **Logística**: tempo de entrega, atraso absoluto e relativo, status do pedido
- **Financeiro**: GMV, frete, forma de pagamento e coerência financeira
- **Cliente**: localização geográfica (estado e cidade)
- **Produto**: categoria principal do pedido
- **Experiência**: nota de avaliação, presença de comentário e tempo de resposta

Além das métricas originais, foram criadas variáveis derivadas e flags analíticas, tais como:
- `delay_bucket`: categorização do atraso em faixas interpretáveis
- `has_review`: indicador de presença de avaliação
- `has_comment`: indicador de avaliação com comentário textual
- `satisfaction_group`: agrupamento simplificado de satisfação (detractor, passive, promoter)
- `is_completed_order` e `has_financials`: flags de consistência operacional

Essas transformações visam reduzir o esforço analítico nas etapas seguintes e garantir consistência nas análises.


In [6]:
con.execute("""
CREATE OR REPLACE VIEW gold_delay_satisfaction AS
WITH order_category AS (
  SELECT
    x.order_id,
    arg_max(pr.product_category_en, x.cnt) AS main_category_en
  FROM (
    SELECT
      order_id,
      product_id,
      COUNT(*) AS cnt
    FROM curated_order_items
    GROUP BY order_id, product_id
  ) x
  LEFT JOIN curated_products pr
    ON x.product_id = pr.product_id
  GROUP BY x.order_id
)
SELECT
  -- chaves
  o.order_id,
  o.customer_id,

  -- status / flags
  o.order_status,
  CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END AS is_completed_order,

  -- review coverage
  CASE WHEN r.order_id IS NULL THEN 0 ELSE 1 END AS has_review,

  -- datas base
  o.purchase_date,
  o.delivered_date,
  o.estimated_delivery_date,

  EXTRACT(year  FROM o.purchase_date) AS purchase_year,
  EXTRACT(month FROM o.purchase_date) AS purchase_month,

  -- logística
  o.days_to_deliver,
  o.delay_days,
  o.delay_days_pos,
  o.early_days_pos,
  o.is_delivered,
  o.is_delayed,

  -- bucket de atraso
  CASE
    WHEN o.order_status <> 'delivered' OR o.delivered_date IS NULL THEN 'not_delivered'
    WHEN o.delay_days_pos = 0 THEN 'on_time'
    WHEN o.delay_days_pos BETWEEN 1 AND 3 THEN 'delay_1_3'
    WHEN o.delay_days_pos BETWEEN 4 AND 7 THEN 'delay_4_7'
    WHEN o.delay_days_pos BETWEEN 8 AND 14 THEN 'delay_8_14'
    ELSE 'delay_15_plus'
  END AS delay_bucket,

  -- itens (GMV / volume)
  COALESCE(i.items_count, 0) AS items_count,
  COALESCE(i.unique_products, 0) AS unique_products,
  COALESCE(i.unique_sellers, 0) AS unique_sellers,
  COALESCE(i.total_freight, 0) AS total_freight,
  COALESCE(i.total_gmv, 0) AS total_gmv,

  -- pagamentos
  COALESCE(p.total_payment_value, 0) AS total_payment_value,
  p.payment_type_main,
  p.max_installments,

  -- coerência financeira
  CASE
    WHEN COALESCE(i.total_gmv, 0) > 0 AND COALESCE(p.total_payment_value, 0) > 0 THEN 1
    ELSE 0
  END AS has_financials,

  -- cliente / geo
  c.customer_state,
  c.customer_city,

  -- categoria principal
  COALESCE(oc.main_category_en, 'unknown') AS main_category_en,

  -- reviews (por pedido)
  r.review_id,
  r.review_score,
  r.has_comment,
  r.is_detractor,
  r.is_promoter,
  r.review_date,
  r.answer_date,
  r.days_to_answer,

  -- grupo de satisfação (legível)
  CASE
    WHEN r.review_score IS NULL THEN 'no_review'
    WHEN r.review_score <= 2 THEN 'detractor'
    WHEN r.review_score = 3 THEN 'passive'
    ELSE 'promoter'
  END AS satisfaction_group

FROM curated_orders o
LEFT JOIN curated_reviews_by_order r
  ON o.order_id = r.order_id
LEFT JOIN curated_items_by_order i
  ON o.order_id = i.order_id
LEFT JOIN curated_payments_by_order p
  ON o.order_id = p.order_id
LEFT JOIN curated_customers c
  ON o.customer_id = c.customer_id
LEFT JOIN order_category oc
  ON o.order_id = oc.order_id;
""")

## Validação da Camada Gold

Após a criação da view `gold_delay_satisfaction`, foram realizadas validações técnicas e analíticas para garantir sua confiabilidade:

- **Granularidade**: verificada unicidade de `order_id`, garantindo 1 linha por pedido
- **Cobertura de dados**:
  - Mais de 99% dos pedidos possuem avaliação
  - Aproximadamente 41% das avaliações possuem comentário textual
- **Distribuição de atraso**: validação dos buckets de atraso, com concentração coerente em pedidos entregues no prazo
- **Sinal analítico**: análise preliminar confirmou forte correlação entre atraso e queda na satisfação do cliente

Essas validações indicam que a camada Gold está consistente, completa e pronta para suportar análises exploratórias e diagnósticas.

In [7]:
# Unicidade
con.execute("""
SELECT
  COUNT(*) AS rows,
  COUNT(DISTINCT order_id) AS distinct_orders,
  COUNT(*) - COUNT(DISTINCT order_id) AS dup_orders
FROM gold_delay_satisfaction;
""").df()

,rows,distinct_orders,dup_orders
0,99441,99441,0


In [8]:
# Cobertura de review
con.execute("""
SELECT
  SUM(has_review) AS orders_with_review,
  COUNT(*) AS total_orders,
  AVG(has_review) AS pct_with_review,
  AVG(CASE WHEN has_review = 1 THEN has_comment ELSE NULL END) AS pct_with_comment_among_reviewed
FROM gold_delay_satisfaction;
""").df()

,orders_with_review,total_orders,pct_with_review,pct_with_comment_among_reviewed
0,98673.0,99441,0.992277,0.413132


In [9]:
# Delay bucket (sanidade)
con.execute("""
SELECT delay_bucket, COUNT(*) AS n
FROM gold_delay_satisfaction
GROUP BY 1
ORDER BY n DESC;
""").df()

,delay_bucket,n
0,on_time,89936
1,not_delivered,2971
2,delay_1_3,1870
3,delay_4_7,1802
4,delay_8_14,1478
5,delay_15_plus,1384


In [10]:
# Relação atraso × satisfação (primeiro sinal)
con.execute("""
SELECT
  delay_bucket,
  AVG(review_score) AS avg_score,
  AVG(is_detractor) AS pct_detractor,
  AVG(is_promoter) AS pct_promoter,
  COUNT(*) AS n
FROM gold_delay_satisfaction
WHERE has_review = 1
GROUP BY 1
ORDER BY n DESC;
""").df()

,delay_bucket,avg_score,pct_detractor,pct_promoter,n
0,on_time,4.290464,0.092640,0.826638,89443
1,not_delivered,1.747631,0.780274,0.143559,2849
2,delay_1_3,3.291037,0.321274,0.531857,1852
3,delay_4_7,2.104691,0.676201,0.225400,1748
4,delay_8_14,1.670816,0.801521,0.109959,1446
5,delay_15_plus,1.722846,0.783521,0.125094,1335


## Estado Final do Ambiente Analítico

A seguir é apresentada a listagem final de tabelas e views disponíveis após a criação da camada Gold.

In [11]:
con.execute("""
SELECT table_schema, table_name, table_type
FROM information_schema.tables
WHERE table_schema NOT IN ('information_schema', 'pg_catalog')
ORDER BY table_type, table_schema, table_name;
""").df()

,table_schema,table_name,table_type
0,main,category_translation,BASE TABLE
1,main,customers,BASE TABLE
2,main,geolocation,BASE TABLE
3,main,order_items,BASE TABLE
4,main,order_payments,BASE TABLE
5,main,order_reviews,BASE TABLE
6,main,orders,BASE TABLE
7,main,products,BASE TABLE
8,main,sellers,BASE TABLE
9,main,curated_category_translation,VIEW


In [12]:
con.close()

## Conclusão da Camada Gold

A camada Gold `gold_delay_satisfaction` cumpre seu papel como base analítica central desta POC, integrando dados operacionais e de experiência do cliente de forma consistente e interpretável.

Os resultados iniciais já indicam um padrão claro:
- Pedidos entregues no prazo apresentam alta taxa de promotores
- Pequenos atrasos geram quedas significativas na satisfação
- Atrasos mais longos e pedidos não entregues estão fortemente associados a avaliações negativas

Com essa base consolidada, a próxima etapa da POC consiste em realizar uma **Análise Exploratória Guiada (EDA)**, com foco em:
- Quantificar o impacto do atraso na satisfação
- Identificar categorias, regiões ou perfis mais sensíveis a atrasos
- Gerar insights acionáveis para priorização de melhorias operacionais

A partir deste ponto, o foco deixa de ser engenharia de dados e passa a ser **interpretação, diagnóstico e tomada de decisão orientada a dados**.